In [10]:
import pandas as pd
from sklearn.metrics import confusion_matrix

MODEL_NAME = "LSOIE"


def get_dev_result(model_name: str):
    model_path = "..\evaluations\\" + model_name + "_dev.txt"
    with open(model_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
        lines = [line.strip().split(" ") for line in lines]
        y_true = [line[1] for line in lines if len(line) == 3]
        y_pred = [line[2] for line in lines if len(line) == 3]
        return y_true, y_pred
y_true, y_pred = get_dev_result(MODEL_NAME)

def get_confusion_matrix(y_true, y_pred):
    labels = list(set(y_true))
    df = pd.DataFrame(
        data=confusion_matrix(y_true, y_pred, labels=labels),
        columns=labels,
        index=labels,
    )
    df.index.name = "Y-True"
    df.columns.name = "Y-Pred"
    return df

#
# Local (metrics per class)
#
df = get_confusion_matrix(y_true, y_pred)

labels = list(set(y_true))
tps = {}
fps = {}
fns = {}
for label in labels:
    tps[label] = df.loc[label, label]
    fps[label] = df[label].sum() - tps[label]
    fns[label] = df.loc[label].sum() - tps[label]

#
# Global
#
micro_averages = {}
macro_averages = {}

correct_predictions = sum(tps.values())

total_predictions = df.values.sum()
accuracy_global = round(correct_predictions / total_predictions,4 ) if total_predictions > 0. else 0.

print("#-- Local measures --#")
print("True Positives:", tps)
print("False Positives:", fps)
print("False Negatives:", fns)

print("\n#-- Global measures --#")
print("Correct predictions:", correct_predictions)
print("Total predictions:", total_predictions)
print("Accuracy:", accuracy_global)

#-- Local measures --#
True Positives: {'B-ARG1': 10, 'B-ARG0': 30, 'O': 542, 'I-ARG0': 38, 'B-V': 22, 'I-ARG1': 20, 'I-V': 0}
False Positives: {'B-ARG1': 35, 'B-ARG0': 26, 'O': 168, 'I-ARG0': 19, 'B-V': 18, 'I-ARG1': 101, 'I-V': 0}
False Negatives: {'B-ARG1': 41, 'B-ARG0': 21, 'O': 125, 'I-ARG0': 46, 'B-V': 29, 'I-ARG1': 36, 'I-V': 69}

#-- Global measures --#
Correct predictions: 662
Total predictions: 1029
Accuracy: 0.6433


In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

      B-ARG0     0.5357    0.5882    0.5607        51
      B-ARG1     0.2222    0.1961    0.2083        51
         B-V     0.5500    0.4314    0.4835        51
      I-ARG0     0.6667    0.4524    0.5390        84
      I-ARG1     0.1653    0.3571    0.2260        56
         I-V     0.0000    0.0000    0.0000        69
           O     0.7634    0.8126    0.7872       667

    accuracy                         0.6433      1029
   macro avg     0.4148    0.4054    0.4007      1029
weighted avg     0.6231    0.6433    0.6287      1029



E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\poetry\venvs\flair-oie-_ETNQf08-py3.9\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
df.style.background_gradient(cmap="Purples")

Y-Pred,B-ARG1,B-ARG0,O,I-ARG0,B-V,I-ARG1,I-V
Y-True,,,,,,,
B-ARG1,10,3,30,0,1,7,0
B-ARG0,1,22,27,1,0,0,0
O,23,12,526,18,20,68,0
I-ARG0,0,13,33,37,0,1,0
B-V,0,0,31,0,19,1,0
I-ARG1,1,1,40,2,0,12,0
I-V,12,1,33,2,7,14,0


In [9]:
from transformers import MarianMTModel, MarianTokenizer, pipeline

pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")
sents = ["The world is very big", "I am a student", "I am a teacher", "I live in Brazil"]
model_name = "Helsinki-NLP/opus-mt-tc-big-en-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
sent = "The Olympic Games take place every four years."
print(pipe(sent))

KeyboardInterrupt: 